In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
import glob
from io import BytesIO

# Задание базовых параметров
batch_size = 32
learning_rate = 0.001
epochs = 10

# Трансформация изображений
transform = transforms.Compose([
    transforms.Resize((224, 224)),           # Приведение к одному размеру
    transforms.RandomHorizontalFlip(),        # Горизонтальное отражение (для увеличения разнообразия)
    transforms.ToTensor(),                    # Преобразование в Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация
])

# Загрузка объединённых данных из папки "consolidated"
full_dataset = datasets.ImageFolder(root='/kaggle/input/autism-image-data/AutismDataset/consolidated', transform=transform)

# Разбиение данных на train, valid и test
train_size = int(len(full_dataset) * 0.7)
valid_size = int(len(full_dataset) * 0.15)
test_size = len(full_dataset) - train_size - valid_size

train_dataset, valid_dataset, test_dataset = random_split(full_dataset, [train_size, valid_size, test_size])

# Создание загрузчиков данных
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Архитектура модели
class AutismClassifier(nn.Module):
    def __init__(self):
        super(AutismClassifier, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten()
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(64*56*56, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

# Перемещаем модель на устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutismClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        running_loss += loss.item()
    accuracy_train = 100 * correct_train / total_train
    print(f"[Эпоха {epoch+1}/{epochs}] Loss: {running_loss:.3f}, Train Accuracy: {accuracy_train:.2f}%")

    # Проверка на валидационном наборе
    model.eval()
    correct_valid = 0
    total_valid = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_valid += labels.size(0)
            correct_valid += (predicted == labels).sum().item()
    accuracy_valid = 100 * correct_valid / total_valid
    print(f"[Эпоха {epoch+1}/{epochs}] Validation Accuracy: {accuracy_valid:.2f}%")

torch.save(model.state_dict(), 'best_model.pth')
print("Модель успешно сохранена!")



In [ ]:
# Загрузка модели из pth-файла
loaded_model = AutismClassifier().to(device)
loaded_model.load_state_dict(torch.load('best_model.pth'))
loaded_model.eval()

correct_test = 0
total_test = 0
predictions = []
ground_truth = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = loaded_model(images)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        ground_truth.extend(labels.cpu().numpy())
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

accuracy_test = 100 * correct_test / total_test
print(f"\nФинальная точность на тестовом наборе: {accuracy_test:.2f}%\n")

# Показываем подробный классификационный отчёт
report = classification_report(np.array(ground_truth), np.array(predictions), digits=4)
print(report)

# Определение аутичных черт по одному фото из локальной ФС

In [ ]:
# Переносим модель на нужное устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutismClassifier().to(device)
model.load_state_dict(torch.load('best_model.pth', map_location=device))
model.eval()

# Загружаем и предобрабатываем изображение
image_path = "/kaggle/input/autism-image-data/AutismDataset/consolidated/Autistic/0015.jpg"
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

image = Image.open(image_path)
image_tensor = transform(image).unsqueeze_(0).to(device)  # Преобразовываем в Tensor и добавляем размер батча

# Прогон через модель
with torch.no_grad():
    output = model(image_tensor)
    probabilities = torch.softmax(output, dim=1)
    pred_class = torch.argmax(probabilities, dim=1)

# Интерпретация результата
classes = ["Non-Autistic", "Autistic"]  # Названия классов зависят от вашей задачи
print(f"Прогноз для изображения '{image_path}' — {classes[pred_class.item()]}. Вероятности: {probabilities.tolist()[0]} ")

# Определение аутичных черт по одному фото из URL

In [ ]:
url = "https://i.pinimg.com/originals/55/b7/93/55b793764f01746f78f54820caa9caeb.jpg"
response = requests.get(url)
image_bytes = response.content

# Преобразуем байты в изображение и далее по схеме
image = Image.open(BytesIO(image_bytes))
image_tensor = transform(image).unsqueeze_(0).to(device)

# Прогон через модель
with torch.no_grad():
    output = model(image_tensor)
    probabilities = torch.softmax(output, dim=1)
    pred_class = torch.argmax(probabilities, dim=1)

# Результат
print(f"Прогноз для изображения по URL '{url}' — {classes[pred_class.item()]}. Вероятности: {probabilities.tolist()[0]} ")

# Определение аутичных черт по множеству фото из локальной ФС

In [ ]:
# Список путей к изображениям
images_paths = glob.glob("/kaggle/input/autism-image-data/AutismDataset/valid/Autistic/*.jpg")

# Преобразование данных
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Пользовательская функция для открытия и преобразования изображений
def load_and_transform(path):
    image = Image.open(path)
    return transform(image)

# Создаем генератор изображений
image_tensors = [load_and_transform(path) for path in images_paths]

# Пакуем в батч
image_batch = torch.stack(image_tensors).to(device)

# Прогон через модель
with torch.no_grad():
    outputs = model(image_batch)
    probabilities = torch.softmax(outputs, dim=1)
    preds = torch.argmax(probabilities, dim=1)

# Выводим прогнозы
for i, path in enumerate(images_paths):
    print(f"Изображение {path}: {classes[preds[i].item()]}, Вероятности: {probabilities[i].tolist()} ")

# 